In [2]:
from tensorflow.keras.optimizers import SGD,Adam
import matplotlib.pyplot as plt



from tensorflow.keras import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, LayerNormalization, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from sklearn.metrics import r2_score
from tensorflow.keras.models import load_model

In [3]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib
import os
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scaler3d2d import preprocess_and_save_scalers,transform_data, inverse_transform_data

In [4]:
import MetaTrader5 as mt5
import numpy as np
import pandas as pd


In [5]:
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)

trade_active = mt5.initialize()

print(trade_active)

if not trade_active:
    print('Initialization failed, check internet connection. You must have Meta Trader 5 installed.')
    mt5.shutdown()

else:
    print(mt5.account_info()._asdict())
    print("\n")
    print(mt5.terminal_info()._asdict())
    print("\n")
    print(mt5.symbols_total())

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4200
True
{'login': 40834668, 'trade_mode': 0, 'leverage': 1000, 'limit_orders': 50, 'margin_so_mode': 0, 'trade_allowed': True, 'trade_expert': True, 'margin_mode': 2, 'currency_digits': 2, 'fifo_close': False, 'balance': 10000.0, 'credit': 0.0, 'profit': 0.0, 'equity': 10000.0, 'margin': 0.0, 'margin_free': 10000.0, 'margin_level': 0.0, 'margin_so_call': 100.0, 'margin_so_so': 50.0, 'margin_initial': 0.0, 'margin_maintenance': 0.0, 'assets': 0.0, 'liabilities': 0.0, 'commission_blocked': 0.0, 'name': 'Demo Standard', 'server': 'Deriv-Demo', 'currency': 'USD', 'company': 'Deriv.com Limited'}


{'community_account': False, 'community_connection': False, 'connected': True, 'dlls_allowed': False, 'trade_allowed': True, 'tradeapi_disabled': False, 'email_enabled': False, 'ftp_enabled': False, 'notifications_enabled': False, 'mqid': True, 'build': 5328, 'maxbars': 100000000, 'codepage': 0, 'ping_last': 250411, '

In [6]:
account = mt5.account_info()
terminal = mt5.terminal_info()

print(account.equity)

if(terminal.connected == True or terminal.trade_allowed == True):
    print("AI is successfully functional")
else:
    print("Please make sure metatrade 5 has internet and algo Trade is Turn On")

10000.0
AI is successfully functional


In [7]:
symbols = mt5.symbols_get()
print(len(symbols))
t_symbol = []
for symbol in symbols:
    if "USD" in symbol.name:
        print("Yes ", symbol.name)
        t_symbol.append(symbol.name)

print(len(t_symbol))

790
Yes  AUDUSD
Yes  EURUSD
Yes  GBPUSD
Yes  NZDUSD
Yes  USDCAD
Yes  USDCHF
Yes  USDJPY
Yes  XAGUSD
Yes  XAUUSD
Yes  XPDUSD
Yes  XPTUSD
Yes  AAVUSD
Yes  ADAUSD
Yes  ALGUSD
Yes  APEUSD
Yes  AVAUSD
Yes  BATUSD
Yes  BCHUSD
Yes  BNBUSD
Yes  BTCUSD
Yes  DOGUSD
Yes  DOTUSD
Yes  DSHUSD
Yes  ETCUSD
Yes  ETHUSD
Yes  FILUSD
Yes  IOTUSD
Yes  LNKUSD
Yes  LTCUSD
Yes  USDCNH
Yes  USDMXN
Yes  USDNOK
Yes  USDPLN
Yes  USDSEK
Yes  USDZAR
Yes  USDHKD
Yes  USDSGD
Yes  USDTHB
Yes  AUDUSDmicro
Yes  EURUSDmicro
Yes  GBPUSDmicro
Yes  NZDUSDmicro
Yes  USDCADmicro
Yes  USDCHFmicro
Yes  USDJPYmicro
Yes  BCHUSD.conv
Yes  ETHUSD.conv
Yes  BTCUSD.conv
Yes  NEOUSD
Yes  SHBUSD
Yes  SOLUSD
Yes  TRXUSD
Yes  UNIUSD
Yes  XLMUSD
Yes  XMRUSD
Yes  XRPUSD
Yes  XTZUSD
Yes  ZECUSD
Yes  LTCUSD.conv
Yes  XALUSD
Yes  XCUUSD
Yes  XNIUSD
Yes  XPBUSD
Yes  XZNUSD
Yes  USD Basket
Yes  AUDUSD DFX 10 Index
Yes  EURUSD DFX 10 Index
Yes  GBPUSD DFX 10 Index
Yes  USDCHF DFX 10 Index
Yes  USDJPY DFX 10 Index
Yes  AUDUSD DFX 20 Index
Yes  EU

In [12]:
market = mt5.copy_rates_from_pos("GBPUSD", mt5.TIMEFRAME_M15, 0, 10000000)
display(market)
print(market.shape)
print("GBPUSD")
t_s = "GBPUSD"

data = []
for i in range(len(market)):
    data.append([market[i][1],market[i][2],market[i][3],market[i][4],market[i][5]])
df = pd.DataFrame(data, columns=["open", "high","low", "close","tick_volume"])
df.to_csv("Generated"+t_s+" dbot.csv", index=False)